In [1]:
import pandas as pd
import ast
import os
from datetime import date, datetime, timedelta
import numpy as np

import geopandas as gpd
import pandas as pd
from dotenv import load_dotenv
from shapely.geometry import Point
from sqlalchemy import create_engine

load_dotenv()

True

In [7]:
load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

df = pd.DataFrame({
    "id":[1, 2, 3],
    "age":[10,20,30]
})

df.to_sql(name="test", con=engine, index=False, if_exists="replace")

C:\Users\add41\AppData\Local\Temp\ipykernel_26988\3911614835.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(name="test", con=engine, index=False, if_exists="replace")


AttributeError: 'Engine' object has no attribute 'cursor'

In [5]:
import pymysql

load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

conn = pymysql.connect(
    host=target_ip,
    port=3306,
    user=username,
    password=password,
    database=db_name,
    charset='utf8mb4'
)

sql = "SELECT * FROM location"

df = pd.read_sql(sql, conn)

df

C:\Users\add41\AppData\Local\Temp\ipykernel_26988\3776114600.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,loc_id,city,district,area,population
0,NTP001,新北市,板橋區,23.14,550256.0
1,NTP002,新北市,三重區,16.32,383035.0
2,NTP003,新北市,中和區,20.14,404507.0
3,NTP004,新北市,永和區,5.71,211631.0
4,NTP005,新北市,新莊區,19.74,421669.0
...,...,...,...,...,...
153,KSH034,高雄市,杉林區,104.00,10595.0
154,KSH035,高雄市,內門區,95.62,12609.0
155,KSH036,高雄市,茂林區,194.00,1838.0
156,KSH037,高雄市,桃源區,928.98,4137.0


In [2]:
def count_hours(time_str: str):
    """根據時間字串計算小時（字串格式為「xx:xx - xx:xx」）"""
    step2 = time_str.replace("–", "-")
    step3 = step2.split("-")
    step4 = list(map(lambda x: x.strip(), step3))

    fmt = "%H:%M"
    start = datetime.strptime(step4[0], fmt)
    end = datetime.strptime(step4[1], fmt)

    delta = end - start
    if delta.days < 0:
        delta += timedelta(days=1)
    hours = delta.total_seconds() / 3600

    return hours

In [3]:
def float_or_int_converter(op_time):
    if np.isnan(op_time) or op_time < 0:
        return 0
    if op_time > 0:
        return op_time
    if op_time == 0:
        return 0

def str_converter(op_time):
    na_words = ["nan", "na", "null", "none", ]

    if op_time == "0" or op_time == "":
        return False

    if op_time.lower() in na_words:
        return False

    return True

def list_converter(op_time):
    op_time = ast.literal_eval(op_time)
    if not isinstance(op_time, list):
        return 0
    else:
        op_hours = 0

        # 營業時間為一週list，採逐日處理
        for day in op_time:
            # 使用分隔符號將"星期X"移除
            step1 = day.split(": ")[1].strip()

            # 若某日為休息則營業時間為0
            if step1 == "休息":
                hours = 0

            elif step1 == "24 小時營業":
                hours = 24

            else:
                # 若營業時間的字串中含有","表示不只一個營業時段
                # 先切割後再逐段處理
                if "," in step1:
                    op_list = step1.split(",")
                    hours = 0
                    for period in op_list:
                        period = period.strip()
                        hours += count_hours(period)
                else:
                    hours = count_hours(step1)

            op_hours += hours

    return op_hours


def trans_op_time_to_hours(op_time):
    """輸入營業時間list，會逐日計算營業時間並加總後回傳
    僅適用google map營業時間格式"""


    # 第一種狀況：輸入為None值
    if pd.isna(op_time):
        return 0

    # 第二種狀況：輸入為浮點數
    if isinstance(op_time, (int, float)):
        return float_or_int_converter(op_time)

    # 第三種情況：輸入為字串
    if isinstance(op_time, (str)):
        if not str_converter(op_time):
            return 0

        else:
            try:
                # 若有營業時間，則逐日計算時數後相加
                return list_converter(op_time)

            except(ValueError, SyntaxError):
                return 0

In [4]:
x = None
y = "0"
z = ""
a = "NaN"
b = np.nan
c = "missing"
d = 0
e = 10
f = -50
g = pd.NA
h = "星期一"

print(trans_op_time_to_hours(x))
print(trans_op_time_to_hours(y))
print(trans_op_time_to_hours(z))
print(trans_op_time_to_hours(a))
print(trans_op_time_to_hours(b))
print(trans_op_time_to_hours(c))
print(trans_op_time_to_hours(d))
print(trans_op_time_to_hours(e))
print(trans_op_time_to_hours(f))
print(trans_op_time_to_hours(g))
print(trans_op_time_to_hours(h))


0
0
0
0
0
0
0
10
0
0
0
